In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from functools import partial
import geopy
import time
from geopy.extra.rate_limiter import RateLimiter
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [2]:
URL = "https://www.caproasia.com/2023/06/21/2023-top-20-most-expensive-cities-in-the-world-top-10-cities-are-new-york-hong-kong-geneva-london-singapore-zurich-san-francisco-tel-aviv-seoul-tokyo/"

In [3]:
# Header for request
HEADER = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en; q=0.5'})
geolocator = Nominatim(user_agent="geocode_locations_from_webpage")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5, max_retries=3)
geo_key = 'e0e73007ebe240c6836e278bbc510da2'
geocoder = OpenCageGeocode(geo_key)

In [4]:
webpage = requests.get(URL, headers=HEADER)
webpage

<Response [200]>

In [5]:
soup = BeautifulSoup(webpage.content, "html.parser")
#soup_4

In [6]:
places = soup.find_all("td", attrs = {'style': 'height: 23px; width: 28.784648%;'})
places

[<td style="height: 23px; width: 28.784648%;">New York</td>,
 <td style="height: 23px; width: 28.784648%;">Hong Kong</td>,
 <td style="height: 23px; width: 28.784648%;">Geneva</td>,
 <td style="height: 23px; width: 28.784648%;">London</td>,
 <td style="height: 23px; width: 28.784648%;">Singapore</td>,
 <td style="height: 23px; width: 28.784648%;">Zurich</td>,
 <td style="height: 23px; width: 28.784648%;">San Francisco</td>,
 <td style="height: 23px; width: 28.784648%;">Tel Aviv</td>,
 <td style="height: 23px; width: 28.784648%;">Seoul</td>,
 <td style="height: 23px; width: 28.784648%;">Tokyo</td>,
 <td style="height: 23px; width: 28.784648%;">Bern</td>,
 <td style="height: 23px; width: 28.784648%;">Dubai</td>,
 <td style="height: 23px; width: 28.784648%;">Shanghai</td>,
 <td style="height: 23px; width: 28.784648%;">Guangzhou</td>,
 <td style="height: 23px; width: 28.784648%;">Los Angeles</td>,
 <td style="height: 23px; width: 28.784648%;">Shenzhen</td>,
 <td style="height: 23px; width:

In [7]:
location_description = geocoder.geocode(places[2].text)
#location_description

In [8]:
city_address = []
for i in range(20):
    
    # GETTING RAW DATA
    try:
        location_description = geocoder.geocode(places[i].text)
    except Exception:
        print(str(i) + " " + "No data found for location: " + places[i].text)
        continue
    
    # GETTING ADDRESS VALUES
    latitude = str(location_description[0]['geometry']['lat'])
    longitude = str(location_description[0]['geometry']['lng'])
    city = places[i].text
    country = location_description[0]['components']['country']
    country_code = location_description[0]['components']['country_code']
    continent = location_description[0]['components']['continent']
    
    #GETTING POSTCODE
    lat_long = str(latitude) + ","+ str(longitude) 
    location2 = geolocator.reverse(lat_long, language="en")
    try:
        postcode = location2.raw['address']['postcode'].replace(" ","")
    except:
        postcode = "NULL"
        
        
    # ADDING VARIABLES TO DICTIONARY
    #print(str(i) +"  "+ city +"  " + country + "  " + country_code.upper() + "  " + continent + "  " + latitude + "  "+ longitude+ "  " + postcode)
    address_dict = {'City': city, 'Country': country, 'Country Code':country_code.upper(), 'Continent': continent, 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode}
    city_address.append(address_dict)
    
print(city_address)

[{'City': 'New York', 'Country': 'United States', 'Country Code': 'US', 'Continent': 'North America', 'Latitude': '40.71', 'Longitude': '-74.01', 'Postcode': '10000'}, {'City': 'Hong Kong', 'Country': 'China', 'Country Code': 'CN', 'Continent': 'Asia', 'Latitude': '22.28', 'Longitude': '114.16', 'Postcode': 'NULL'}, {'City': 'Geneva', 'Country': 'Switzerland', 'Country Code': 'CH', 'Continent': 'Europe', 'Latitude': '46.2', 'Longitude': '6.15', 'Postcode': '1204'}, {'City': 'London', 'Country': 'United Kingdom', 'Country Code': 'GB', 'Continent': 'Europe', 'Latitude': '51.51', 'Longitude': '-0.13', 'Postcode': 'SW1A2DX'}, {'City': 'Singapore', 'Country': 'Singapore', 'Country Code': 'SG', 'Continent': 'Asia', 'Latitude': '1.36', 'Longitude': '103.82', 'Postcode': '578774'}, {'City': 'Zurich', 'Country': 'Switzerland', 'Country Code': 'CH', 'Continent': 'Europe', 'Latitude': '47.37', 'Longitude': '8.54', 'Postcode': '8001'}, {'City': 'San Francisco', 'Country': 'United States', 'Country

In [9]:
ca_most_expensive_df = pd.DataFrame.from_dict(city_address)
ca_most_expensive_df

,City,Country,Country Code,Continent,Latitude,Longitude,Postcode
0,New York,United States,US,North America,40.71,-74.01,10000
1,Hong Kong,China,CN,Asia,22.28,114.16,NULL
2,Geneva,Switzerland,CH,Europe,46.2,6.15,1204
3,London,United Kingdom,GB,Europe,51.51,-0.13,SW1A2DX
4,Singapore,Singapore,SG,Asia,1.36,103.82,578774
5,Zurich,Switzerland,CH,Europe,47.37,8.54,8001
6,San Francisco,United States,US,North America,37.78,-122.42,94102
7,Tel Aviv,Israel,IL,Asia,32.09,34.78,6296802
8,Seoul,South Korea,KR,Asia,37.57,126.98,04523
9,Tokyo,Japan,JP,Asia,35.68,139.77,103-8230
